In [ ]:
import os
import shutil

from PIL import Image

In [ ]:
dataset_pca_dir = '' # input dir containing PCA-reduced groups of 5 slices per scan
dataset_txt_dir = '' # input dir containing radiological reports

output_dir = '' # output dir for slices
temp_output_file = '' # temporary output file for reports
output_file = '' # output file for reports

os.makedirs(output_dir, exist_ok=True)

In [ ]:
output_lines = []

for folder in sorted(os.listdir(dataset_pca_dir)):
    folder_pca_path = os.path.join(dataset_pca_dir, folder)
    folder_txt_path = os.path.join(dataset_txt_dir, folder)
    if os.path.isdir(folder_pca_path) and os.path.isdir(folder_txt_path):
        image_files = sorted([f for f in os.listdir(folder_pca_path) if f.endswith('.jpg')])
        txt_files = [f for f in os.listdir(folder_txt_path) if f.endswith('.txt')]
        if len(txt_files) != 1:
            print(f"Error: There are {len(txt_files)} .TXT files in folder {folder_txt_path}.")
            continue

        txt_file_path = os.path.join(folder_txt_path, txt_files[0])

        with open(txt_file_path, 'r') as file:
            report = file.read().strip()

        for image_file in image_files:
            old_image_path = os.path.join(folder_pca_path, image_file)
            new_image_name = f"{folder}_{image_file}"
            new_image_path = os.path.join(output_dir, new_image_name)

            shutil.copy2(old_image_path, new_image_path)

            with Image.open(old_image_path) as img:
                grayscale_img = img.convert("L")
                grayscale_img.save(new_image_path)

            output_lines.append(f"{new_image_name} {report}")

with open(temp_output_file, 'w') as file:
    file.write("\n".join(output_lines))

with open(temp_output_file, 'r') as infile, open(output_file, 'w') as outfile:

    for line in infile:
        parts = line.split(maxsplit=1)
        if len(parts) == 2:
            image_name_without_extension = os.path.splitext(parts[0])[0]
            outfile.write(f"{image_name_without_extension}    {parts[1]}")
        else:
            outfile.write(line)

os.remove(temp_output_file)

print(f"File {output_file} created with success and slices copied in {output_dir}.")